In [1]:
!pip install transformers
!pip install sentence_transformers
!pip install datasets
!pip install accelerate
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [33]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.2 MB/s eta 0:00:00


In [26]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM ,TrainingArguments, AutoTokenizer , DataCollatorForSeq2Seq,Seq2SeqTrainer , Trainer , MarianMTModel, MarianTokenizer
from datasets import Dataset , load_dataset,load_metric
import tensorflow as tf
import json
import torch
from tqdm import tqdm
import ast

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer , model = model,return_tensors=  'tf')

In [14]:
data = pd.read_csv(r'/content/Book12.csv')

In [15]:
data

,عربي,English
0,شهد العالم الاسلامي أواخر القرن الخامس عشر و ب...,The Islamic world witnessed an intense conflic...
1,دولة المماليك قامت في مصر و الشام و الحجاز و ج...,"The Mamluk state was established in Egypt, the..."
2,وتحطم الاسطول المملوكي عام 1509م في معركة ديو ...,", and the destruction of the Mamluk fleet in 1..."
3,الدولة العثمانية مع بداية القرن السادس عشر الم...,The Ottoman Empire: At the beginning of the si...
4,الدولة الصفوية و بعد ان تمكن السلطان سليم الاو...,The Safavid state. After Sultan Selim I was ab...
...,...,...
221,اجتمعت عوامل كثيرة ساهمت في اندلاع ثورة 1919م،...,Many factors came together that contributed to...
222,زيادة نمو الوعي الوطني لدى المصريين من أجل الا...,Increased growth in national awareness among E...
223,الإجراءات العنيفة التي اتخذتها سلطات الاحتلال ...,Violent measures taken by the occupation autho...
224,تدعي إنجلترا وحلفاؤها أنهم يقاتلون من أجل حرية...,England and its allies claim that they are fig...


In [16]:
tokenizer('اهلا انا محمد')

{'input_ids': [17037, 30, 842, 7434, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [17]:
data = Dataset.from_pandas(data)
data = data.train_test_split(.1)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['عربي', 'English'],
        num_rows: 203
    })
    test: Dataset({
        features: ['عربي', 'English'],
        num_rows: 23
    })
})

In [ ]:
data['train'][:1]

{'عربي': ['الحملة الفرنسية على الشام 1799م:الأسباب: تحالفت الدولة العثمانية مع إنجلترا وروسيا لطرد الفرنسيين من مصر بالقوة العسكرية، فأعدوا حملتين لغزو مصر، الأولى بحرية على جزيرة رودس إلى مصر من الشمال، والثانية برية إلى مصر من الشرق عبر بلاد الشام.'],
 'English': ['The French campaign against the Levant, 1799 AD: Reasons: The Ottoman Empire allied with England and Russia to expel the French from Egypt by military force, so they prepared two campaigns to invade Egypt, the first freely on the island of Rhodes to Egypt from the north, and the second overland to Egypt from the east via the Levant.']}

In [ ]:
data['train'][0]

{'عربي': 'الحملة الفرنسية على الشام 1799م:الأسباب: تحالفت الدولة العثمانية مع إنجلترا وروسيا لطرد الفرنسيين من مصر بالقوة العسكرية، فأعدوا حملتين لغزو مصر، الأولى بحرية على جزيرة رودس إلى مصر من الشمال، والثانية برية إلى مصر من الشرق عبر بلاد الشام.',
 'English': 'The French campaign against the Levant, 1799 AD: Reasons: The Ottoman Empire allied with England and Russia to expel the French from Egypt by military force, so they prepared two campaigns to invade Egypt, the first freely on the island of Rhodes to Egypt from the north, and the second overland to Egypt from the east via the Levant.'}

In [ ]:
len(data['train'])

203

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['عربي', 'English'],
        num_rows: 203
    })
    test: Dataset({
        features: ['عربي', 'English'],
        num_rows: 23
    })
})

In [ ]:
max_input =256
max_output= 256
def process(example):
  # print(example['عربي'][0])
  inputs = [ss for ss in example['عربي']]
  target = [ss for ss in example['English']]
  model_inputs = tokenizer(inputs , max_length=256 , truncation=True)
  with tokenizer.as_target_tokenizer():
   labels = tokenizer(target , max_length=max_output , truncation=True)
  model_inputs['labels'] = labels ['input_ids']
  return model_inputs

In [ ]:
tokenized_data = data.map(process, batched=True,remove_columns=data['train'].column_names)

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 203
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 23
    })
})

In [ ]:
tokenizer.decode(tokenized_data['train']['input_ids'][3])

'اتجه العثمانيين الي الي مصر و الشام بسبب رغبة السلطان سليم الاول في توسيع ممتلكاته في الشرق عن طريق الاستيلاء علي مصر التي تمثل قلب العالم الاسلامي و ايضا بسبب ادراك السلطان سليم الاول ان الصدام مع الدولة الاوروبية</s>'

In [ ]:
# process(data['train'])

In [ ]:
# dataes = []
# for i in range(17):
#   answer_str = data['train']['Translation'][i]
#   datas = json.loads(answer_str.replace("'", "\""))
#   dataes.append(datas)

In [ ]:
# dataes[0]['en']

In [ ]:
data_collator = DataCollatorForSeq2Seq(model = model  ,tokenizer = tokenizer)

In [ ]:
tokenized_data['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 203
})

In [ ]:
training_args = TrainingArguments('finetuned_model',learning_rate = 2e-5, per_device_train_batch_size=32,
                                      num_train_epochs  = 20 ,
                                      evaluation_strategy = 'epoch')

In [ ]:
trainer = Trainer(model=model, args=training_args
                         ,train_dataset = tokenized_data['train']
                         ,eval_dataset = tokenized_data['test']
                         ,data_collator = data_collator
                         ,tokenizer = tokenizer)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.610756
2,No log,1.527933
3,No log,1.443931
4,No log,1.374197
5,No log,1.319512
6,No log,1.275993
7,No log,1.238244
8,No log,1.205847
9,No log,1.178291
10,No log,1.154274


TrainOutput(global_step=20, training_loss=0.85272216796875, metrics={'train_runtime': 521.5018, 'train_samples_per_second': 0.652, 'train_steps_per_second': 0.038, 'total_flos': 8193870397440.0, 'train_loss': 0.85272216796875, 'epoch': 20.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/trained_models/translation_model.h5')

In [11]:
tokenizer = MarianTokenizer.from_pretrained("/content/drive/MyDrive/trained_models/translation_model.h5")
model = MarianMTModel.from_pretrained("/content/drive/MyDrive/trained_models/translation_model.h5")

input_text = '''لماذا عانت الدولة الصفوية'''

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate translations
output = model.generate(input_ids)

# Decode the generated output
translated_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [12]:
print(translated_text)

Why did the Safavid state suffer?


In [20]:
data

DatasetDict({
    train: Dataset({
        features: ['عربي', 'English'],
        num_rows: 203
    })
    test: Dataset({
        features: ['عربي', 'English'],
        num_rows: 23
    })
})

In [48]:
source_texts = data['test']['عربي']
reference_texts = [[ref] for ref in data['test']['English']]

In [49]:
def generate_translations_in_batches(model, tokenizer, texts, batch_size=32, max_length=256):
    translations = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs)
        batch_translations = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        translations.extend(batch_translations)
    return translations

In [50]:
translated_texts = generate_translations_in_batches(model, tokenizer, source_texts)


100%|██████████| 1/1 [00:49<00:00, 49.52s/it]


In [53]:
metric = load_metric("sacrebleu")
results = metric.compute(predictions=translated_texts, references=reference_texts)

In [54]:
results['score']

46.913806214651466

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save_pretrained('/content/drive/MyDrive/trained_models')